In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
planilha = pd.ExcelFile('Precificação WEG-OIW. Outubro V4.xlsx')
tabPrecificacao1 = pd.read_excel(planilha, 'precificação - OIW')
tabPrecificacao2 = pd.read_excel(planilha, 'precificação - WEG')

# cada aba em um dataframe
precificacaoDF1 = tabPrecificacao1[['Geração','Número de módulos','Inversor: Qtde.',' Inversor: Potência AC (kW) ','KIT OIW']]
precificacaoDF2 = tabPrecificacao2[['Geração','Número de módulos','Inversor: Qtde.',' Inversor: Potência AC (kW) ','KIT WEG (sem %)']]

c:\Users\Lukas\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\Lukas\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
precificacaoDF1

,Geração,Número de módulos,Inversor: Qtde.,Inversor: Potência AC (kW),KIT OIW
0,688.300,10,1,5kw - growatt,13200.180
1,757.130,11,1,5kw - growatt,14051.500
2,825.960,12,1,5kw - growatt,14902.820
3,1032.450,15,1,5kw - growatt,17893.620
4,1170.110,17,1,8kw - growatt,20859.003
5,1514.260,22,1,8kw - growatt,25552.443
6,1651.920,24,1,8kw - growatt,27255.083
7,2340.220,34,1,15kw - growatt,39812.680
8,2477.880,36,1,15kw - growatt,41515.320
9,3166.180,46,1,15kw - growatt,51339.040


In [4]:
# verificar em cada linha a potencia do inversor - de 0 até 10 = monofasico - acima de 10 = trifasico
precificacaoDF1['Potencia Inversores'] = ''
inversorPotenciaDF = precificacaoDF1[' Inversor: Potência AC (kW) '].str.split('kw', expand=True)
inversorPotenciaDF[0] = pd.to_numeric(inversorPotenciaDF[0])
inversorPotenciaDF.loc[inversorPotenciaDF[0] <= 10, 1] = 'Monofasico'
inversorPotenciaDF.loc[inversorPotenciaDF[0] > 10, 1] = 'Trifasico'
precificacaoDF1['Potencia Inversores'] = inversorPotenciaDF[1]

# adicionando colunas e modificando colunas
precificacaoDF1['Marca Inversores'] = 'Growatt'
precificacaoDF1['potenciaModulos'] = 460
precificacaoDF1['potenciaInversores'] = inversorPotenciaDF[0]
precificacaoDF1
precificacaoDF1.rename(columns={'Geração':'kwh_mes', 'Número de módulos':'quantidadeModulos', 'Inversor: Qtde.':'quantidadeInversores',
       ' Inversor: Potência AC (kW) ':'modeloInversor1', 'KIT OIW':'preco_fibrocimento', 'Marca Inversores':'marcaInversores',
       'Potencia Inversores':'ligacaoInversores'}, inplace=True)

# reorganizando colunas
precificacaoDF1 = precificacaoDF1[['potenciaModulos','kwh_mes','quantidadeModulos','quantidadeInversores','modeloInversor1','marcaInversores','ligacaoInversores','potenciaInversores','preco_fibrocimento']]

C:\Users\Lukas\AppData\Local\Temp\ipykernel_14800\420362525.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precificacaoDF1['Potencia Inversores'] = ''
C:\Users\Lukas\AppData\Local\Temp\ipykernel_14800\420362525.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precificacaoDF1['Potencia Inversores'] = inversorPotenciaDF[1]
C:\Users\Lukas\AppData\Local\Temp\ipykernel_14800\420362525.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [5]:
from distutils import errors


precificacaoDF1[['kwh_mes','preco_fibrocimento']] = precificacaoDF1[['kwh_mes','preco_fibrocimento']].apply(pd.to_numeric, errors='ignore')
precificacaoDF1.dtypes

potenciaModulos           int64
kwh_mes                 float64
quantidadeModulos         int64
quantidadeInversores      int64
modeloInversor1          object
marcaInversores          object
ligacaoInversores        object
potenciaInversores        int64
preco_fibrocimento      float64
dtype: object

In [5]:
# formatando para comando sql

listString = []
listSQLCommand = []
for row in precificacaoDF1.values:
    listString = []
    for column in row:
        # verificando se eh numero
        if isinstance(column, int) or isinstance(column, float):
            listString.append(str(column))
        else:
            listString.append('"' + str(column) + '"')
    listSQLCommand.append(','.join(listString))

listSQLCommand = ['(' + x + ')' for x in listSQLCommand]
listSQLCommand  


['(460,688.3,10,1,"5kw - growatt","Growatt","Monofasico",5,13200.18)',
 '(460,757.13,11,1,"5kw - growatt","Growatt","Monofasico",5,14051.5)',
 '(460,825.96,12,1,"5kw - growatt","Growatt","Monofasico",5,14902.82)',
 '(460,1032.45,15,1,"5kw - growatt","Growatt","Monofasico",5,17893.620000000003)',
 '(460,1170.11,17,1,"8kw - growatt","Growatt","Monofasico",8,20859.002800000002)',
 '(460,1514.26,22,1,"8kw - growatt","Growatt","Monofasico",8,25552.4428)',
 '(460,1651.92,24,1,"8kw - growatt","Growatt","Monofasico",8,27255.0828)',
 '(460,2340.22,34,1,"15kw - growatt","Growatt","Trifasico",15,39812.68)',
 '(460,2477.88,36,1,"15kw - growatt","Growatt","Trifasico",15,41515.32)',
 '(460,3166.18,46,1,"15kw - growatt","Growatt","Trifasico",15,51339.04)',
 '(460,3510.33,51,1,"20kw - growatt","Growatt","Trifasico",20,56784.479999999996)',
 '(460,3647.99,53,1,"20kw - growatt","Growatt","Trifasico",20,58923.96)',
 '(460,3992.14,58,1,"20kw - growatt","Growatt","Trifasico",20,63617.4)',
 '(460,4336.29,63